# PDE-based opinion model: Numerical simulation
## Hiroki Sayama
### sayama@binghamton.edu

This is the code used to simulate the PDE-based opinion formation model studied in the following paper:
- Sayama, H. (2020) _Phys. Rev. E_ 102, 012303. https://doi.org/10.1103/PhysRevE.102.012303

Note that this simulation is quite time-consuming (a single run may take several hours). Once the simulation ends, the results will be saved in a CSV file and the final state of the system will be plotted. A space-time visualization as shown in the paper can be generated using Mathematica (code available on GitHub).

In [ ]:
const n = 1000;
const L = 50;
const dx = L / n;
const dy = L / n;
const dt = 0.001;
const R = Int64(20 / dy);

const Ph = 1.;
const c = 1.;
const d = 0.2;

In [ ]:
function g(y::Float64, σ::Float64, μ::Float64)::Float64
    1/2μ * 1/(sqrt(2π)*σ) * (exp(-1/2*((y-μ)/σ)^2) - exp(-1/2*((y+μ)/σ)^2))
end;

In [ ]:
function bound(i::Int64)::Int64
    mod(i - 1, n) + 1
end;

In [ ]:
function update(P, σ::Float64, μ::Float64)
    nP = zeros(n)
    for i in 1:n
        gradP = (P[bound(i+1)] - P[bound(i-1)]) / (2dx);
        lapP = (P[bound(i+1)] + P[bound(i-1)] - 2*P[i]) / (dx^2);
        GP = sum(P[bound(i+j)]*g(j*dy, σ, μ) for j in -R:R)*dy;
        gradGP = sum((P[bound(i+j+1)] - P[bound(i+j-1)])*g(j*dy, σ, μ) for j in -R:R) / 2;
        nP[i] = P[i] + (d*lapP - c*(gradP*GP + P[i]*gradGP))*dt;
    end
    nP
end;

In [ ]:
function simulation(σ, μ)
    P = (Ph - 0.01)*ones(n) + 0.02*rand(n);

    results = [];
    push!(results, P);

    t = 0.
    while t < 100
        t += dt
        P = update(P, σ, μ)
        if true in [isnan(pp) for pp in P]
            println("NaN occurred!")
            push!(results, P)
            return results
        end
        if floor(t) != floor(t - dt)
            print(floor(Int, t), " ")
            push!(results, P)
        end
    end
    return results
end;

In [ ]:
using DelimitedFiles

In [ ]:
σ = 1.;
μ = 1.;
results = simulation(σ, μ);
writedlm("results_sigma=" * string(σ) * "_mu=" * string(μ) * ".csv", results, ",");

In [ ]:
using Plots

In [ ]:
plot(0:dx:dx*(n-1), results[end], xlabel = "x", ylabel = "P(x)", 
     title = "Final state", ylims = (0, 20))